<a href="https://colab.research.google.com/github/pansplawik/eegParkinsonCognitiveDisorders/blob/master/secondVersion.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install mne

Importowanie pakietów

In [2]:
import mne
import numpy as np
import matplotlib.pyplot as plt
import os
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC
import csv
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score
from mne.decoding import PSDEstimator

Ściezki plików

In [3]:
labels = pd.read_csv('/content/drive/MyDrive/dyplom/dataset.csv', sep=';')
path='/content/drive/MyDrive/dyplom/EEG'

In [4]:
def load_epochs_from_dir(path, labels):
    epochs_array = []
    for filename, label in zip(os.listdir(path), labels['PD']):
        print(filename)
        if filename.endswith('.edf'):
            filepath = os.path.join(path, filename)
            raw = mne.io.read_raw_edf(filepath, preload=True)
            annotations = raw.annotations
            change=0
            for i in range(len(annotations)):
              if annotations.description[i] == "break":
                annotations.description[i] = "BAD" 
                change=change+1
              elif annotations.description[i] == 'Break end':
                annotations.description[i] = "BAD"
                change=change+1
              elif annotations.description[i] == 'Miesnie':
                annotations.description[i] = "BAD"
                change=change+1
              elif annotations.description[i] == 'Break start':
                annotations.description[i] = "BAD"
                change=change+1
              elif annotations.description[i] == 'Ruch':
                annotations.description[i] = "BAD"
                change=change+1
            print(change)
            raw.set_annotations(annotations)
            raw.filter(1, 40, fir_design='firwin')
            events = mne.make_fixed_length_events(raw, duration=1.0)
            epochs = mne.Epochs(raw, events, tmin=0, tmax=2, baseline=(0,2), detrend=1, reject_by_annotation='BAD',preload=True)
            epochs_array.append([epochs,label])
            print()
            print(epochs.drop_log)
            del epochs
            del raw
            del annotations
    return epochs_array


In [7]:
def psds(epochs_array, labels):
    data = []
    for epochs, label in zip(epochs_array, labels):
        epo_spectrum = epochs.compute_psd(method='multitaper', fmin=5, fmax=30, picks='eeg')
        psds, freqs = epo_spectrum.get_data(return_freqs=True)
        for p in psds:
            data.append([p, label])
        del psds
        del freqs
    return data

In [ ]:
data = load_epochs_from_dir('/content/drive/MyDrive/dyplom/EEG',labels)
df = pd.DataFrame(data, columns=['value', 'label'])
df.to_csv('data.csv', index=False)

In [ ]:
psd = psds([epochs for epochs, label in data], [label for epochs, label in data])
df = pd.DataFrame(psd, columns=['psds', 'label'])
df.to_csv('psd.csv', index=False)

In [9]:
X_train, X_test, y_train, y_test = train_test_split([x[0] for x in psd], [x[1] for x in psd], test_size=0.2, random_state=42)
X_train_2d = np.array([x[0] for x in X_train]).reshape(len(X_train), -1)
X_test_2d = np.array([x[0] for x in X_test]).reshape(len(X_test), -1)
y_train=np.array(y_train)
y_train=y_train.transpose()
y_test=np.array(y_test)
y_test=y_test.transpose()

In [10]:
print(X_test_2d.shape)
print(X_train_2d.shape)
svm = SVC(kernel='linear', C=1, random_state=42)
svm.fit(X_train_2d, y_train)

(25876, 50)
(103502, 50)


SVC(C=1, kernel='linear', random_state=42)

In [11]:
y_pred = svm.predict(X_test_2d)
from sklearn.metrics import accuracy_score
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)

Accuracy: 0.4390554954397898


In [12]:
from joblib import dump, load
dump(svm, 'trzeci.joblib') 

['trzeci.joblib']